## This script is used to infer CNV pattern in HD samples.

In [1]:
# import packages
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, ListedColormap, BoundaryNorm
import seaborn as sns

from scipy.sparse import csr_matrix
import scipy.sparse as sp
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list, fcluster
import os,sys,re
from tqdm import tqdm
import time
import infercnvpy as cnv
import random
import gc
import scipy.io as spio

In [ ]:
# read sample information
meta = pd.read_csv("HD-OV 100.csv", index_col=False)
sampleName = meta['sampleName']

## inferCNV

In [ ]:
## smooth count matrix

for subdirectory in sampleName:

    adata = sc.read_h5ad(f"{subdirectory}/outs/adata_8um.h5ad")
    adata.obs['in_tissue'] = adata.obs['in_tissue'].astype(float)
    adata.obs['array_row'] = adata.obs['array_row'].astype(float)
    adata.obs['array_col'] = adata.obs['array_col'].astype(float)
    adata.obsm['spatial'] = adata.obsm['spatial'].astype(float)
    adata.X = adata.layers['counts'].copy()

    whole_obs = adata.obs
    cancer_obs = whole_obs[whole_obs["annotations"]=="Malignant"]
    modify_X = adata.X.todense()

    for i in range(0,len(cancer_obs)):
        
        row_idx = cancer_obs['array_row'][i]
        col_idx = cancer_obs['array_col'][i]

        infer_obs = cancer_obs[(cancer_obs['array_row']==row_idx) & (cancer_obs['array_col']==col_idx)]

        neighbor_cancer_obs = cancer_obs[(cancer_obs['array_row'].between(row_idx-1, row_idx+1)) & (cancer_obs['array_col'].between(col_idx-1, col_idx+1))]
        neighbor_cancer_obs = neighbor_cancer_obs[~((neighbor_cancer_obs['array_row'] == row_idx) & (neighbor_cancer_obs['array_col'] == col_idx))]

        if len(neighbor_cancer_obs) > 0:
            raw_X = adata[infer_obs.index].X
            index_to_update = whole_obs.index.get_loc(infer_obs.index[0])
            neibor_cancer_X = adata[neighbor_cancer_obs.index].X

            raw_exp = raw_X.todense().A1
            neibor_cancer_exp = np.mean(adata[neighbor_cancer_obs.index].X, axis=0).A1

            modify_exp = 0.75*raw_exp + 0.25*neibor_cancer_exp
            modify_X[index_to_update,:] = modify_exp

    sparse_matrix = sp.csr_matrix(modify_X)
    adata.X = sparse_matrix

    adata.write_h5ad(f"{subdirectory}/outs/adata_8um_smooth.h5ad")

In [ ]:
# Run CNV analysis for each sample
for subdirectory in sampleName:

    print("=================== process =============", subdirectory)

    # read adata file
    adata = sc.read_h5ad(f"{subdirectory}/outs/adata_8um_smooth.h5ad")
    adata.obs['in_tissue'] = adata.obs['in_tissue'].astype(float)
    adata.obsm['spatial'] = adata.obsm['spatial'].astype(float)
    adata.obs['array_row'] = adata.obs['array_row'].astype(float)
    adata.obs['array_col'] = adata.obs['array_col'].astype(float)

    # annotation the chromosome and position of each gene
    gene_order = pd.read_csv(f"{subdirectory}/ref/geneEx-h38_human_genes_pos.txt", sep="\t", header=None, index_col=0)
    gene_order.columns = ["chromosome", "start", "end"]
    adata.var = pd.merge(adata.var, gene_order, left_index=True, right_index=True)

    # remove NA
    adata = adata[: , adata.var.chromosome.notna()] 

    # delete duplicated index
    duplicate_index = adata.var.index.duplicated()
    adata = adata[:, ~duplicate_index]

    # sort by chromosome and start
    adata.var = adata.var.sort_values(by=['chromosome','start'])

    # remove IGKC and IGHG1
    adata = adata[:, ~adata.var_names.isin(['IGKC','IGHG1'])].copy()

    # normalize the data
    sc.pp.log1p(adata)
    
    # run infercnv
    cnv.tl.infercnv(
        adata,
        reference_key="annotations",
        reference_cat=['Fibroblast', 'Endothelial', 'Macrophage',
                        'DC', 'Neutrophil', 'T', 'NK', 'Mast', 'Others'],
        window_size=190,
        step=1,
        n_jobs=32,
    )

    # randomly select 5000 cells to draw the cnv heatmap
    num_cells = 5000
    all_cells = list(adata.obs_names)
    selected_cells = random.sample(all_cells, num_cells)
    subset_adata = adata[selected_cells]
    cnv.pl.chromosome_heatmap(
        subset_adata, groupby="annotations", dendrogram=True)
    plt.savefig(f"/{subdirectory}_chromosome_on_all_cells.pdf".format(subdirectory), bbox_inches='tight')
    del subset_adata
    gc.collect()

    # annotate the cnv status
    adata.obs["cnv_status"] = "normal"
    adata.obs.loc[adata.obs["annotations"].isin(["Malignant"]), "cnv_status"] = (
        "tumor"
    )
    
    # save the result of the all cells
    adata.obs['in_tissue'] = adata.obs['in_tissue'].astype(str)
    adata.obsm['spatial'] = adata.obsm['spatial'].astype(str)
    adata.obs['array_row'] = adata.obs['array_row'].astype(str)
    adata.obs['array_col'] = adata.obs['array_col'].astype(str)
    adata.write_h5ad(
        f"/{subdirectory}/adata_infercnv.h5ad".format(subdirectory))

## a. Clusters

In [ ]:
# Run CNV analysis for each sample
for subdirectory in sampleName:

    print("=================== process =============", subdirectory)

    # read adata file
    adata = sc.read_h5ad(f"/{subdirectory}/adata_infercnv.h5ad")
    adata.obs['in_tissue'] = adata.obs['in_tissue'].astype(float)
    adata.obsm['spatial'] = adata.obsm['spatial'].astype(float)
    adata.obs['array_row'] = adata.obs['array_row'].astype(float)
    adata.obs['array_col'] = adata.obs['array_col'].astype(float)
    
    # choose malignant cells
    bdata = adata[adata.obs["cnv_status"] == "tumor", :]

    # clustering the malignant cells
    cnv.tl.pca(bdata, n_comps=20)
    cnv.pp.neighbors(bdata)
    cnv.tl.leiden(
        bdata,
        resolution=0.6,
        random_state=0,
        n_iterations=2,
        directed=False,
    )

    # plot the cnv heatmap of the malignant cells
    cnv.pl.chromosome_heatmap(bdata, groupby="cnv_leiden", show=False)
    plt.savefig(f"/{subdirectory}_chromosome_on_malignant.pdf".format(
        subdirectory), dpi=300, bbox_inches='tight')
    
    # plot the spatial distribution of the malignant cells
    sc.pl.spatial(bdata, color='cnv_leiden', img_key=None, show=False)
    plt.savefig(f"/{subdirectory}_spatial_cnv.pdf".format(
        subdirectory), dpi=300, bbox_inches='tight')

    # save the result of the malignant cells
    bdata.obs['in_tissue'] = bdata.obs['in_tissue'].astype(str)
    bdata.obsm['spatial'] = bdata.obsm['spatial'].astype(str)
    bdata.obs['array_row'] = bdata.obs['array_row'].astype(str)
    bdata.obs['array_col'] = bdata.obs['array_col'].astype(str)
    bdata.write_h5ad(
        f"/{subdirectory}/filter_malig_cnv.h5ad")

## b. Scores

In [ ]:
# compute CNV score of malignant cell clusters
col_names = []

for subdirectory in sampleName:
    print("=================== process =============", subdirectory)
    
    # read adata file
    adata = sc.read_h5ad(f"/{subdirectory}/filter_malig_cnv.h5ad")
    adata.obs['in_tissue'] = adata.obs['in_tissue'].astype(float)
    adata.obsm['spatial'] = adata.obsm['spatial'].astype(float)
    adata.obs['array_row'] = adata.obs['array_row'].astype(float)
    adata.obs['array_col'] = adata.obs['array_col'].astype(float)

    # compute the mean of cnv score of all cells in each cnv_leiden
    cnv_leiden_clusters = np.unique(adata.obs['cnv_leiden'])
    results = []
    for cluster in cnv_leiden_clusters:
        col_names.append(subdirectory + '_' + str(cluster))
        cluster_indices = np.where(adata.obs['cnv_leiden'] == cluster)[0]
        cluster_cnv = adata.obsm['X_cnv'][cluster_indices]
        cluster_sum_normalized = np.mean(cluster_cnv, axis=0)
        results.append(cluster_sum_normalized)
    
    # combine the X matrix
    X = np.transpose(np.concatenate(np.asarray(results), axis=0))
    X_total = np.concatenate((X_total, X), axis=1)

    cnv_leiden_clusters = np.unique(adata.obs['cnv_leiden'])
    for cluster in cnv_leiden_clusters:
        col_names.append(f'{subdirectory}_' + str(cluster))

In [ ]:
# get the row names
target_chromosomes = ['chrM', 'chrX', 'chrY']
row_names = adata.var_names[~np.isin(adata.var['chromosome'], target_chromosomes)] #行名

# create and restore the combined matrix
cnv = pd.DataFrame(X_total, index=row_names, columns=col_names)
cnv.to_csv('CNV_sum_filter.csv')

## c. Levels

In [ ]:
for subdirectory in sampleName:

    # read adata file
    adata = sc.read_h5ad(f"{subdirectory}/adata_infercnv.h5ad")
    adata.obs['in_tissue'] = adata.obs['in_tissue'].astype(float)
    adata.obsm['spatial'] = adata.obsm['spatial'].astype(float)
    adata.obs['array_row'] = adata.obs['array_row'].astype(float)
    adata.obs['array_col'] = adata.obs['array_col'].astype(float)
    adata = adata[adata.obs['annotations'].isin(['Malignant', 'Epithelial'])]

    # define cnv levels
    min_value = np.min(adata.obs['cnv_score'])
    max_value = np.max(adata.obs['cnv_score'])
    q1 = np.percentile(adata.obs['cnv_score'], 25)
    q2 = np.percentile(adata.obs['cnv_score'], 50)
    q3 = np.percentile(adata.obs['cnv_score'], 75)
    bounds = [min_value, q1, q2, q3, max_value]
    interval_labels = ['Bottom', 'Lower-Mid', 'Upper-Mid', 'Top']
    adata.obs['cnv_score_interval'] = pd.cut(adata.obs['cnv_score'], bins=bounds, labels=interval_labels, include_lowest=True)

    # the color of each level
    colors = ['#781c68', '#b79ad8',  '#ffce94', '#ff9c2b']
    cmap = ListedColormap(colors)
    norm = BoundaryNorm(bounds, cmap.N)

    # plot
    sc.pl.spatial(adata, color='cnv_score_interval', palette=colors, img_key=None, show=False, size=1.4, frameon=False)
    plt.savefig(f"/{subdirectory}_score_interval_malig.tiff", format="tiff", bbox_inches='tight')

## d. States

In [ ]:
# read the CNV score matrix
cnv = pd.read_csv("CNV_sum_filter.csv", index_col=0, header=0)

In [ ]:
# transpose the DataFrame so that the sample name becomes the row and the gene becomes the column
cnv_transposed = cnv.transpose()

# compute the variance of each gene
df = cnv_transposed
gene_variances = df.var(axis=0)

# choose top 2000 high variance genes
high_var_genes = gene_variances.nlargest(2000).index

# select the high variance genes
merge_df_transposed = cnv_transposed.loc[:, high_var_genes]

In [ ]:
# perform hierarchical clustering
Z = linkage(merge_df_transposed, method='ward')

# plot the hierarchical clustering dendrogram
plt.figure(figsize=(30, 5), dpi=500)
dendrogram(Z, labels=merge_df_transposed.index)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Sample')
plt.ylabel('Distance')
plt.show()

In [ ]:
# choose a distance threshold
distance_threshold = 2.4

# use fcluster to group based on distance threshold
clusters = fcluster(Z, t=distance_threshold, criterion='distance')

# add the clustering result to the original DataFrame
merge_df_transposed['Cluster'] = clusters

# plot the hierarchical clustering dendrogram
plt.figure(figsize=(40,5))
dendrogram(Z, labels=merge_df_transposed.index, color_threshold=distance_threshold)
plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('Sample')
plt.ylabel('Distance')
plt.axhline(y=distance_threshold, color='r', linestyle='--')
plt.show()

In [ ]:
# save the clustering result
cluster = merge_df_transposed[['Cluster']]
cluster.to_csv(f"cnv_clu5_th2.4.csv")

## e. Similarity

In [ ]:
# import packages
from sklearn.metrics import pairwise_distances

In [ ]:
# read the wes data
wes_all = pd.read_csv("./ref/wes_cn.csv", sep=",")

In [ ]:
cosinesimilarity_df = pd.DataFrame()

# calculate the cosine similarity of each sample
for variable in sampleName:
    print(variable)

    # get wes name of the variable sample
    wesName = meta[meta['sampleName'] == variable]['rnaID'].astype(str).str.cat(sep=',')
    wesName = re.sub(r'(\d+)R(\d+)(?![\d(])', r'\1W\2', re.sub(r'\(.*\)', '', wesName))

    if wesName in wes_all.columns:

        # read the adata file and get the HD CNV matrix
        adata = sc.read_h5ad(f"{wesName}/filter_malig_cnv.h5ad")
        adata = adata[:,~adata.var["chromosome"].isin(['chrM','chrX','chrY'])]
        X_cnv = adata.obsm['X_cnv'][:, [i for i, gene_id in enumerate(adata.var['gene_ids']) if gene_id in wes_all['gene_id'].values]]
        hd_cnv = np.asarray(np.mean(X_cnv, axis=0))

        # read the wes data and gete the WES CNV matrix
        wes = wes_all[['gene_id', wesName]].copy()
        wes = wes[wes.gene_id.isin(adata.var["gene_ids"])].copy()
        wes = wes.set_index('gene_id').loc[adata[:,adata.var["gene_ids"].isin(wes.gene_id)].var["gene_ids"]].reset_index()
        wes_cnv = wes[wesName] - 2
        wes_cnv = wes_cnv.to_numpy().reshape(1, -1)

        # compute the cosine similarity
        score = 1 - pairwise_distances(X=wes_cnv, Y=hd_cnv, metric="cosine").flatten()
        
        # add the result to the DataFrame
        cosinesimilarity_df.loc[variable, 'cosinesimilarity'] = score[0]

In [ ]:
# save the result
meta1 = meta.set_index('sampleName')
df = pd.concat([cosinesimilarity_df , meta1['Tumor_Type']], axis=1)
df.to_csv('./cosinesimilarity_all.csv')